#### This notebook is for resizing the push notification base to get an specific daily consumtion

In [1]:
import sys, os 
prj_path = os.path.dirname(os.getcwd())
utils_path = prj_path + '/utils'
if  not utils_path in sys.path:
    print('adding utils to path ')
    sys.path.insert(1, utils_path)

adding utils to path 


In [2]:
from utilities import *
from google_cloud import BigQuery, Storage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

import os
import sys

import boto3
import base64

#Cliente BigQuery
AUTH_BIGQUERY = base64.b64decode(os.environ['SECRET_AUTH_BIGQUERY_MODEL'])
bq = BigQuery(AUTH_BIGQUERY)
storage =Storage(AUTH_BIGQUERY)

# Load config.
files = glob.glob(f"{utils_path}/*.yml") 
conf = load_conf(files)

In [11]:
today = pd.to_datetime('today')
bop = (today - timedelta(days = today.day -1)).strftime('%Y-%m-%d')

In [ ]:
bop = '2022-04-01'

eop = '2022-04-30'
site = 'MLM'
#We need to call this from somewhere.
budget = 3000000

In [ ]:
df = consumtion_expand(site = site, bop = bop,eop = eop)

In [ ]:
#some params
N = df.shape[0]
avg_fi = budget/(N)
lim_sup = 3*avg_fi

In [ ]:
#Check for the daily consumption for tomorrow.
month_seq = df.pay_created_dt
fi = []
for date in month_seq:
    #Calculations only until date = eop -1
    if date < pd.to_datetime(eop):
        #BOP
        if date == pd.to_datetime(bop):
            F = 0
        else:
            yesterday = date - timedelta(days = 1)
            I = df.pay_created_dt == yesterday
            F =  df.loc[I].ACC_AMOUNT.values[0]
            
        f = daily_consumption(budget = budget,
                              eop = eop,
                              date = date,
                              Fj = F,
                              verbose = False)
        fi.append(f)
    #EOP dont need estimation.
    else:
        fi.append(budget- df.ACC_AMOUNT.to_list()[-1])
    # ALARMS        
    T_level = 1-((pd.to_datetime(eop) - date).days/N)
    T_level = np.round(T_level,2)
    B_level = F/budget
    B_level = np.round(B_level,2)
    if f > lim_sup:
        logger.info(f" date :{date} - Need a daily consumption over the limit 3B/N = {lim_sup}")
    if (T_level >.5) & (B_level<.5):
        logger.info(f" date :{date} - Check for an under consumtion %B: {B_level}, %T:{T_level}.")
    if (T_level<.5 - B_level>.5):
        logger.info(f" date :{date} - Check for an over consumtion %B: {B_level}, %T:{T_level}.")

df['forecast_consumption'] = [i for i in fi]

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.stairs(df.forecast_consumption, label = 'updated daily consumption f(i+1)')
ax.hlines(y = avg_fi, xmin=0,xmax = N,linestyle = '--',color = 'g', label = 'average daily budget')
ax.hlines(y = lim_sup, xmin=0,xmax = N, linestyle = '--',color = 'r',label = 'daily spending limit')
ax.stairs(df.AMOUNT, label = 'real consumtion (fi)')
ax.legend(loc = 'upper left')

In [ ]:
yesterday = (pd.to_datetime('today') - timedelta(days = 1)).strftime('%Y-%m-%d')

In [ ]:
df.loc[df.pay_created_dt==pd.to_datetime(yesterday)]

#### Relacion SENT vs Consumo.

In [ ]:
%%time
#Al momento necesito es c = f(#push). No requiero el budget.
bop = '2022-01-01'
eop = '2022-04-05'
df = consumtion_expand(site = site, bop = bop,eop = eop)

# Hoy manny push.
q = QueryParamatrized(query_= 'HOW_MANNY_PUSH', query_param = {'bop':bop ,'eop':eop, 'site':site}, conf = conf)
sent = bq.execute_response(q)
sent['sent_date'] = pd.to_datetime(sent.sent_date)

#cupon variation.
q = QueryParamatrized(query_= 'CUPON_VARIATION', query_param = {'bop':bop ,'eop':eop, 'site':site}, conf = conf)
cupon_value = bq.execute_response(q)
cupon_value['pay_created_dt'] = pd.to_datetime(cupon_value.pay_created_dt)
cupon_value.sort_values(by = 'pay_created_dt', inplace= True)

#dataset
push_cost = df[['pay_created_dt','AMOUNT']].merge(sent,right_on = ['sent_date'],left_on =['pay_created_dt'],how = 'left')
push_cost = push_cost.merge(cupon_value[['pay_created_dt','mkt_cpn_amount']],on = 'pay_created_dt', how = 'left')
push_cost = push_cost[['pay_created_dt','AMOUNT','sent','mkt_cpn_amount']]
push_cost['day_of_week'] = [x.weekday() for x in push_cost.pay_created_dt]

push_cost.sent.fillna(0, inplace = True)
push_cost.mkt_cpn_amount.fillna(method = 'ffill', inplace=True)

In [ ]:
#Time series of push
fig, (ax1,ax2) = plt.subplots(1,2,figsize = (18,5))
ax1.stem(push_cost.pay_created_dt,push_cost.sent)
ax1.set_ylabel('# push')
ax1.set_xlabel('day')
ax1.grid()
ax2.hist(push_cost.sent, bins = 40)
ax2.grid()

In [ ]:
#Time series of push
fig, (ax1, ax2) = plt.subplots(1,2,figsize = (18,5))
ax1.stem(push_cost.pay_created_dt,push_cost.AMOUNT)
ax1.set_ylabel('daily consumption')
ax1.set_xlabel('day')
ax1.grid()

ax2.hist(push_cost.AMOUNT, bins = 40)
ax2.grid()

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.stem(push_cost.pay_created_dt,push_cost.mkt_cpn_amount)
ax.set_ylabel('cupon value')
ax.set_xlabel('day')
ax.grid()

In [ ]:
#Relation N push = f(daily cosnumtion)
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.scatter(push_cost.AMOUNT,push_cost.sent, s = [x.month*20 for x in push_cost.pay_created_dt])
ax.set_ylabel('# push')
ax.set_xlabel('daily consumption')
ax.vlines(x = avg_fi, ymin=0,ymax = np.max(push_cost.sent),linestyle = '--',color = 'g', label = 'average daily budget - march')
ax.vlines(x = lim_sup, ymin=0,ymax = np.max(push_cost.sent), linestyle = '--',color = 'r',label = 'daily spending limit - march')
ax.hlines(y = np.percentile(push_cost.sent,q = 50), xmin=0,xmax = lim_sup, linestyle = '-',color = 'k',label = 'median # sent push')

ax.hlines(y = np.percentile(push_cost.sent,q = 5), xmin=0,xmax = lim_sup, linestyle = '--',color = 'k',label = 'sent push p5%')
ax.hlines(y = np.percentile(push_cost.sent,q = 95), xmin=0,xmax = lim_sup, linestyle = '--',color = 'k',label = 'sent push p95%')

ax.legend()

In [ ]:
#clean before traing
push_cost_filter = push_cost.copy()
push_cost_filter.loc[push_cost_filter.sent <= 5000,'sent'] = 0
push_cost_filter.loc[push_cost_filter.sent == 0,'AMOUNT'] = 0

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.scatter(push_cost_filter.AMOUNT,push_cost_filter.sent, s = [x.month*20 for x in push_cost_filter.pay_created_dt])
ax.set_ylabel('# push')
ax.set_xlabel('daily consumption')
ax.vlines(x = avg_fi, ymin=0,ymax = np.max(push_cost_filter.sent),linestyle = '--',color = 'g', label = 'average daily budget - march')
ax.vlines(x = lim_sup, ymin=0,ymax = np.max(push_cost_filter.sent), linestyle = '--',color = 'r',label = 'daily spending limit - march')
ax.hlines(y = np.percentile(push_cost.sent,q = 50), xmin=0,xmax = lim_sup, linestyle = '-',color = 'k',label = 'median # sent push')

ax.hlines(y = np.percentile(push_cost.sent,q = 5), xmin=0,xmax = lim_sup, linestyle = '--',color = 'k',label = 'sent push p5%')
ax.hlines(y = np.percentile(push_cost.sent,q = 95), xmin=0,xmax = lim_sup, linestyle = '--',color = 'k',label = 'sent push p95%')

ax.legend()

In [ ]:
#Get a model.
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import HuberRegressor, LinearRegression

target_var = 'sent'
time_var = 'pay_created_dt'
model_vars = set(push_cost_filter.columns) - set([target_var,time_var])
print(model_vars)

rb = RobustScaler()
push_cost_filter_ = pd.DataFrame(
    rb.fit_transform(
        push_cost_filter[model_vars]),
    columns=push_cost_filter[model_vars].columns
)

X_train, X_test, y_train, y_test = train_test_split(push_cost_filter_[model_vars],
                                                    push_cost_filter[target_var],
                                                    random_state= 142,
                                                    shuffle=True,
                                                    test_size=0.33)

In [ ]:
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [50,80, 90, 100, 110],
    'max_features': [1,2],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [10, 50, 100, 200]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search_rf = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1,verbose = -1)
grid_search_rf.fit(X_train,y_train.ravel())


In [ ]:
param_grid = {
    'epsilon': [1.35,5,10,50],
    'alpha': [10**-4,10**-3,10**-2]
}

huber = HuberRegressor()
grid_search_huber = GridSearchCV(estimator = huber, param_grid = param_grid, cv = 3, n_jobs = -1,verbose = -1)
grid_search_huber.fit(X_train,y_train.ravel())

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train.ravel())

In [ ]:
rf_sc = grid_search_rf.best_estimator_.score(X_test,y_test)
hu_sc = grid_search_huber.best_estimator_.score(X_test,y_test)
lm_sc = lm.score(X_test,y_test)
print(rf_sc)
print(hu_sc)
print(lm_sc)

In [ ]:
def mse(y_true,y_pred):
    return np.mean((y_true-y_pred)**2)

In [ ]:
error = []
m = {'rf':grid_search_rf.best_estimator_,
     'hubber':grid_search_huber.best_estimator_,
     'lm': lm
    }

for model_type,model in m.items():       
    y_pred = model.predict(X_test)
    error.append(mse(y_test,y_pred))

In [ ]:
error

In [ ]:
pd.to_pickle(grid_search_huber,'huber.pkl')
pd.to_pickle(rb,'scaler.pkl')

In [ ]:
model = grid_search_huber.best_estimator_

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
#Relation N push = f(daily cosnumtion)
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.scatter(y_test,y_pred, label = '(y_test,y_pred)')
ax.plot(y_test,y_test ,label = 'prefect prediction')
ax.set_xlabel('y_test')
ax.set_ylabel('y_pred')
ax.grid()

In [ ]:
X_test.columns

In [ ]:
#Supongamos que necesito el limite superior dado un cupon de 200.
syntetic_data = pd.DataFrame({
    'day_of_week':[0,1,2,3,4,5,6], 
    'AMOUNT':[lim_sup,lim_sup,lim_sup,lim_sup,lim_sup,lim_sup,lim_sup], 
    'mkt_cpn_amount':[200,200,200,200,200,200,200]}
)
syntetic_data = pd.DataFrame(rb.transform(syntetic_data),columns=syntetic_data.columns)
syntetic_data

In [ ]:
cut = np.floor(lm.predict(syntetic_data))

In [ ]:
cut

In [ ]:
plt.hist(y_test - y_pred, bins = 20)

#### Check.

(F(t=i)+(n-i)*fi+1) = B

ACC_AMOUNT + fi+1*(n-1)

In [ ]:
(0 + 30*100000.0)/budget

In [ ]:
(16800.0 + 29*102868.96551724138)/budget

In [ ]:
(1497750.0+ 1502250.0)/budget

In [ ]:
backbone

In [ ]:
1.502250e+06+1576550.0

In [ ]:
#Ver como varia el budget de una campaña.
#Ver como varian los incrementales.
#Ver como ranquea el modelo y comparar con los incrementales reales.
#Ver si hay componenete estacional.

In [ ]:
budget = 3000000
eop_ = pd.to_datetime(eop) - timedelta(days = 1)
month_seq = pd.date_range(start = bop, end = eop_,freq = 'D')
fj = []
cj = []
Fj = []
N = ((pd.to_datetime(eop) - pd.to_datetime(bop)).days)
avg_fi = 3*budget/N
k = 0
for k, date in enumerate(month_seq) :
           
    if date == pd.to_datetime(bop):
        print('------start campain------')
        fi[k] = budget/N
        if  consumo_nb.loc[consumo_nb.pay_created_dt == date].empty:
            print(f'No consumption for {date}')
            Fi[k] = 0
            ci[k] = 0
        else:
            Fi[k] = consumo_nb.loc[consumo_nb.pay_created_dt == date].AMOUNT.values
            ci[k] = Fi[k]
        print('-------------------------')
    else:
         if  consumo_nb.loc[consumo_nb.pay_created_dt == date].empty:
            print(f'No consumption for {date}')
            I = (consumo_nb.pay_created_dt < date)&(~consumo_nb.ACC_AMOUNT.isnull())
            Fi[k] = consumo_nb.loc[I].ACC_AMOUNT.values[-1]
            ci = 0
        else:
            Fi = consumo_nb.loc[consumo_nb.pay_created_dt == date].ACC_AMOUNT.values
            ci = consumo_nb.loc[consumo_nb.pay_created_dt == date].AMOUNT.values
        
        
    fii = daily_consumption(budget = budget,
                            eop = eop,
                            bop = bop,
                            date_ = date,
                            Fi = Fi,
                            ci = ci,
                            fi = fi[k])
    k = k+1
    fi[k] =fii


#### Budget por campaña

In [ ]:
%%time
buget_variation = bq.execute_response(""" 
SELECT 
    push.CAMPAIGN_ID,
    budget.MKT_TOOL_ID,
    budget.BUDGET_LC,
    min(push.SENT_DATE) minDate_push_event,
    max(push.SENT_DATE) maxDate_push_event,
    min(budget.TIM_MONTH) minDate_budget
FROM meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT push 
LEFT JOIN meli-marketing.PUSH.LK_PUSH_BUDGET_COUPON budget 
ON push.CAMPAIGN_ID = budget.PUSH_CAMPAIGN_ID
AND budget.SIT_SITE_ID = 'MLB'
WHERE 1=1
AND BATCH_ID LIKE 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%'
AND push.SENT_DATE BETWEEN DATE '2022-01-01' AND DATE '2022-03-30'
GROUP BY 1,2,3
ORDEr BY 2,3""")

In [ ]:
buget_variation.head(4)

#### Push events and model prediction.

In [ ]:
#model
storage.downlad_file(gs_source_path = 'gs://marketing-modelling/ML/NB/MLB/model/model_MLB.pkl', 
                     destination_file = 'old_model_nb.pkl')
model = pd.read_pickle('old_model_nb.pkl')

In [ ]:
%%time
q = pre.fun_query_apply('2022-01-05',PAIS ='MLB')
df = bq.execute_response(q)

In [ ]:
df.head()

In [ ]:
df['score'] = model.predict_proba(df)

In [ ]:
%%time
site = 'MLB'
date_intial = '2022-01-01'
date_end = '2022-03-30'
push_population = bq.execute_response(f"""
    WITH push as ( 
            SELECT 
                push.CUS_CUST_ID,
                push.SIT_SITE_ID,
                push.CAMPAIGN_ID,
                push.SENT_DATE, 
                push.EVENT_TYPE 
            FROM meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT push 
            WHERE BATCH_ID LIKE 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%' 
                AND push.EVENT_TYPE in ('sent','control') 
                AND push.APP = 'mercadolibre'
                AND push.sit_site_id = '{site}' 
                AND push.SENT_DATE >= DATE '{date_intial}' AND push.SENT_DATE <= DATE '{date_end}' 
             ),
    orders_ as (
        SELECT 
            push.CUS_CUST_ID, 
            push.CAMPAIGN_ID,
            push.SENT_DATE, 
            push.EVENT_TYPE,
            orders.ORD_CREATED_DT,
            ROW_NUMBER() OVER (PARTITION BY push.CUS_CUST_ID ORDER BY orders.ORD_CREATED_DT) AS ORDERING_PURCHASE
        FROM push LEFT JOIN meli-bi-data.WHOWNER.BT_ORD_ORDERS orders
        ON push.CUS_CUST_ID = orders.ORD_BUYER.ID 
            AND push.sit_site_id = orders.SIT_SITE_ID 
            AND ORD_CLOSED_DT is not null
            AND orders.ord_status = 'paid' 
            AND orders.ORD_CLOSED_DT BETWEEN push.SENT_DATE AND push.SENT_DATE + 4 
            AND orders.ORD_CREATED_DT = push.SENT_DATE
        )

    SELECT 
        CUS_CUST_ID,
        CAMPAIGN_ID,
        SENT_DATE,
        EVENT_TYPE,
        CASE WHEN ORD_CREATED_DT is null THEN 0 ELSE 1 END AS CONVERSION
    FROM orders_
    WHERE ORDERING_PURCHASE = 1
    """ 
                                     )

In [ ]:
push_population.head(3)

In [ ]:
#model
storage.downlad_file(gs_source_path = 'gs://marketing-modelling/ML/NB/MLB/model/model_MLB.pkl', 
                     destination_file = 'old_model_nb.pkl')
model = pd.read_pickle('old_model_nb.pkl')

In [ ]:
model = pd.read_pickle('old_model_nb.pkl')

In [ ]:
model.predict_proba()

In [ ]:
a = 0 
B=100
delta = 12
for i in np.arange(2,30,1):
    a += 1/(i-1)

In [ ]:
(B-delta)*a